In [13]:
import os 
import pandas as pd 
import numpy as np

In [14]:
def removeSpaces():
    # List all files in the current working directory
    files = os.listdir("data")
    i = 1
    for file in files:
        #new_name = file.replace(" ","_")
        os.rename(file, f' {i}')
        i += 1

In [130]:
def readErpSummary(file):
    df = pd.read_excel(file,sheet_name=2)
    unit_code = df['UNIT CODE'][0]
    print(df)
    
    df["TOTAL_100%"] = df["CAT"] + df["EXAM"]

    distinction_list = df[df['TOTAL_100%'].between(69, 101)]
    credit_list = df[df['TOTAL_100%'].between(59, 69)]
    pass_list = df[df['TOTAL_100%'].between(49,59)]
    
    supplementary_list = df[df['TOTAL_100%'].between(39,50)]
    retake_list = df[df['TOTAL_100%'].between(-1,40)]

    # Customize the export settings
    custom_header = ['distinction', 'credit', 'pass', 'supplementary','retake']
    count_data = {
    'distinction': [len(distinction_list)],
    'credit':[ len(credit_list)],
    'pass': [len(pass_list)],
    'supplementary':[len(supplementary_list)],
    'retake': [len(retake_list)]
    } 
    
    statitstics = pd.DataFrame(count_data)
    
    #log to excel file info
    with pd.ExcelWriter(f'./processed/{unit_code}output.xlsx',mode='w') as excel_writer:
        statitstics.to_excel(excel_writer, sheet_name='Sheet1', index=False)
        
        if(len(retake_list) > 0):
            retake_students_info = pd.DataFrame(retake_list)
            retake_students_info.to_excel(excel_writer, sheet_name='Sheet2', index=False)
        
        if(len(supplementary_list) > 0):
            supplementary_Students_info = pd.DataFrame(supplementary_list)
            supplementary_Students_info.to_excel(excel_writer, sheet_name='Sheet3', index=False)
    
    print(statitstics)

In [134]:

dir_path = 'data/'
print(os.getcwd())
x = 1 
for filename in os.listdir():     
                #readErpSummary(file_path)
            if(os.path.isfile(filename)):
                newfilename = f'unit_{x}.xlsx'
                os.rename(filename, newfilename)
                
                #call this functions 
                readErpSummary(newfilename)

                x = x+1

/home/frank/Desktop/WEB/projects/examsum/data
        STUDENT_NO         TRIMESTER  UNIT CODE   CAT  EXAM
0  CIT-3483-3/2023  TRIMESTER 3 2023  ITCR  055  30.0  43.0
1  CIT-3182-3/2023  TRIMESTER 3 2023  ITCR  055  30.0  38.0
2  CIT-3178-3/2023  TRIMESTER 3 2023  ITCR  055   NaN   NaN
   distinction  credit  pass  supplementary  retake
0            1       1     0              0       0


KeyError: 'UNIT CODE'